# Functional Responses

**How well do different mathematical models, e.g., based upon foraging theory (mechanistic) principles vs. phenomenological ones, fit to functional responses data across species?**

Functional responses:
  * Are the relationships between a consumer's biomass consumption rate and abundance of a target resource. 

  * Arise from fundamental biological and physical constraints on consumer-resource interactions.

  * Determine the rate of biomass flow between species in ecosystems across the full scale of sizes. 

  * Play a key role in determining the stability of food webs that underpin ecosystems

NB: Two main fields of interest are **N_TraitValue** (the number of resources consumed per consumer per unit time) and **ResDensity** (the resource abundance). 

Each ID value corresponds to one curve and one individual functional response curve

In [26]:
# Some im
ports to explore the datasets
import pandas as pd
import scipy as sc
import matplotlib.pylab as pl
import seaborn as sns
import numpy as np
import csv

SyntaxError: invalid syntax (<ipython-input-26-863f17194166>, line 2)

## Getting Started

In [0]:
data = pd.read_csv("../data/CRat.csv")
print("Loaded {} columns.".format(len(data.columns.values)))

In [0]:
print(data.columns.values)

#### Making new data frame with only columns of interest for initial plots

In [0]:
newdata = data[['ID','ResDensity','N_TraitValue']].copy()
print(newdata)

Check if there are any columns with NA values

In [0]:
newdata2 = newdata[newdata.isna().any(axis=1)]
print(newdata2)

#### Removing IDs with less than 5 data points

In [0]:
newdata.groupby('ID').ID.count() # grouping data points by ID number and storing a count of instances

Setting a threshold number of repeats. If a ID has less than the threshold number of repeats, all of its data is replaced with null values

In [0]:
threshold = 5
ValueCounts = newdata['ID'].value_counts()
toremove = ValueCounts[ValueCounts <= threshold].index
newdata.replace(toremove, np.nan, inplace=True)

In [0]:
newdata['ID'].value_counts() #Counting number of repeated IDs 

Remove all rows with NA data

In [0]:
newdata_no_missing = newdata.dropna()

In [0]:
newdata_no_missing.isnull().sum() #count number of NAs in data set to be sure

#### Save modified data into a .csv file

In [0]:
newdata_no_missing.to_csv('../data/FunResData.csv')

## NLLS Fitting Script

In [0]:
print(newdata)